In [12]:
import pandas as pd

rating = pd.read_csv('../data/movies_data/ratings_movies.csv')
rating.head(3)

,Unnamed: 0,userId,movieId,rating,date,title,genres
0,0,1,1,4.0,2000-07-30 18:45:03,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,1,3,4.0,2000-07-30 18:20:47,Grumpier Old Men (1995),Comedy|Romance
2,2,1,6,4.0,2000-07-30 18:37:04,Heat (1995),Action|Crime|Thriller


In [13]:
# выделить из признака title год выпуска фильма
import re 
def get_year_release(arg):
    #находим все слова по шаблону "(DDDD)"
    candidates = re.findall(r'\(\d{4}\)', arg) 
    # проверяем число вхождений
    if len(candidates) > 0:
        #если число вхождений больше 0, очищаем строку от знаков "(" и ")"
        year = candidates[0].replace('(', '')
        year = year.replace(')', '')
        return int(year)
    else:
        #если год не указан, возвращаем None
        return None

In [21]:
# Создайте в таблице новый признак year_release, который соответствует году выпуска фильма.
# У скольких фильмов не указан год их выпуска?
rating['year_release'] = rating['title'].apply(get_year_release)
rating[rating['year_release'].notna() == False].shape[0]

18

In [33]:
# Какой фильм, выпущенный в 1999 году, получил наименьшую среднюю оценку зрителей?
rating[rating['year_release'] == 1999].groupby('title')['rating'].mean().sort_values().index[0]

'Bloodsport: The Dark Kumite (1999)'

In [34]:
# Какое сочетание жанров фильмов (genres), выпущенных в 2010 году, получило наименьшую среднюю оценку (rating)?
rating[rating['year_release'] == 2010].groupby('genres')['rating'].mean().sort_values().index[0]

'Action|Sci-Fi'

In [35]:
# Какой пользователь (userId) посмотрел наибольшее количество различных (уникальных) комбинаций жанров (genres) фильмов?
rating.groupby('userId')['genres'].nunique().sort_values(ascending=False).index[0]

599

In [44]:
# Найдите пользователя, который выставил наименьшее количество оценок, но его средняя оценка фильмам наибольшая.
rating.groupby('userId')['rating'].agg(['count', 'mean']).sort_values(by=['count', 'mean'], ascending=[True, False]).index[0]

53

In [50]:
# Найдите сочетание жанров (genres) за 2018 году, которое имеет наибольший средний рейтинг (среднее по столбцу rating), 
# и при этом число выставленных ему оценок (количество значений в столбце rating) больше 10.
table = rating[rating['year_release'] == 2018].groupby('genres')['rating'].agg(['count', 'mean'])
table[table['count']>10].sort_values('mean',ascending=False)

,count,mean
genres,,
Action|Adventure|Sci-Fi,14,3.928571
Action|Comedy|Sci-Fi,12,3.875000


In [120]:
# Создайте сводную таблицу, которая иллюстрирует зависимость среднего рейтинга фильма от года выставления оценки и жанра.
rating['year_rating'] = pd.to_datetime(rating['date']).dt.year
pivot = rating.pivot_table(values='rating', index='year_rating', columns='genres', aggfunc='mean')
display(pivot.head(2))

# Проверьте гипотезы
# A За весь период (с 1996 по 2018 год) сочетание жанров Action|Adventure ни разу не получало среднюю оценку ниже 3.
print(pivot[(pivot.index>=1996)]['Action|Adventure'].min()>=3)

# B Наилучшую оценку жанр Action|Adventure|Animation|Children|Comedy|IMAX получил в 2010 году.
print(pivot['Action|Adventure|Animation|Children|Comedy|IMAX'].sort_values(ascending=False).index[0] == 2010)

# C Среди сочетаний жанров, получивших наивысшую среднюю оценку в 2018 году, есть сочетание Animation|Children|Mystery.
pivot_2018 = pivot[pivot.index==2018]
print(pivot_2018['Animation|Children|Mystery'].iloc[0] == pivot_2018.max().max())

# D Для жанра Comedy прослеживается тенденция падения рейтинга с каждым годом (с 1996 по 2018).
serie_comedy = pivot[(pivot.index>=1996)]['Comedy']
print(all(serie_comedy.index == serie_comedy.sort_values(ascending=False).index))

genres,(no genres listed),Action,Action|Adventure,Action|Adventure|Animation,Action|Adventure|Animation|Children,Action|Adventure|Animation|Children|Comedy,Action|Adventure|Animation|Children|Comedy|Fantasy,Action|Adventure|Animation|Children|Comedy|IMAX,Action|Adventure|Animation|Children|Comedy|Romance,Action|Adventure|Animation|Children|Comedy|Sci-Fi,...,Romance|Thriller,Romance|War,Romance|Western,Sci-Fi,Sci-Fi|IMAX,Sci-Fi|Thriller,Sci-Fi|Thriller|IMAX,Thriller,War,Western
year_rating,,,,,,,,,,,,,,,,,,,,,
1996,NaN,2.730769,3.454545,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,2.666667,NaN,3.838095,NaN,3.117647
1997,NaN,3.538462,4.150000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.400000,NaN,3.923077,NaN,3.000000


True
False
True
False
